In [2]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


## Notebook magic

In [3]:
from IPython.core.magic import Magics, magics_class, line_cell_magic
from IPython.core.magic import cell_magic, register_cell_magic, register_line_magic
from IPython.core.magic_arguments import argument, magic_arguments, parse_argstring
import subprocess
import os

In [4]:
@magics_class
class PyboardMagic(Magics):
    @cell_magic
    @magic_arguments()
    @argument('-skip')
    @argument('-unix')
    @argument('-pyboard')
    @argument('-file')
    @argument('-data')
    @argument('-time')
    @argument('-memory')
    def micropython(self, line='', cell=None):
        args = parse_argstring(self.micropython, line)
        if args.skip: # doesn't care about the cell's content
            print('skipped execution')
            return None # do not parse the rest
        if args.unix: # tests the code on the unix port. Note that this works on unix only
            with open('/dev/shm/micropython.py', 'w') as fout:
                fout.write(cell)
            proc = subprocess.Popen(["../micropython/ports/unix/build-2/micropython-2", "/dev/shm/micropython.py"], 
                                    stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            print(proc.stdout.read().decode("utf-8"))
            print(proc.stderr.read().decode("utf-8"))
            return None
        if args.file: # can be used to copy the cell content onto the pyboard's flash
            spaces = "    "
            try:
                with open(args.file, 'w') as fout:
                    fout.write(cell.replace('\t', spaces))
                    printf('written cell to {}'.format(args.file))
            except:
                print('Failed to write to disc!')
            return None # do not parse the rest
        if args.data: # can be used to load data from the pyboard directly into kernel space
            message = pyb.exec(cell)
            if len(message) == 0:
                print('pyboard >>>')
            else:
                print(message.decode('utf-8'))
                # register new variable in user namespace
                self.shell.user_ns[args.data] = string_to_matrix(message.decode("utf-8"))
        
        if args.time: # measures the time of executions
            pyb.exec('import utime')
            message = pyb.exec('t = utime.ticks_us()\n' + cell + '\ndelta = utime.ticks_diff(utime.ticks_us(), t)' + 
                               "\nprint('execution time: {:d} us'.format(delta))")
            print(message.decode('utf-8'))
        
        if args.memory: # prints out memory information 
            message = pyb.exec('from micropython import mem_info\nprint(mem_info())\n')
            print("memory before execution:\n========================\n", message.decode('utf-8'))
            message = pyb.exec(cell)
            print(">>> ", message.decode('utf-8'))
            message = pyb.exec('print(mem_info())')
            print("memory after execution:\n========================\n", message.decode('utf-8'))

        if args.pyboard:
            message = pyb.exec(cell)
            print(message.decode('utf-8'))

ip = get_ipython()
ip.register_magics(PyboardMagic)

## pyboard

In [57]:
import pyboard
pyb = pyboard.Pyboard('/dev/ttyACM0')
pyb.enter_raw_repl()

In [9]:
pyb.exit_raw_repl()
pyb.close()

In [58]:
%%micropython -pyboard 1

import utime
import ulab as np

def timeit(n=1000):
    def wrapper(f, *args, **kwargs):
        func_name = str(f).split(' ')[1]
        def new_func(*args, **kwargs):
            run_times = np.zeros(n, dtype=np.uint16)
            for i in range(n):
                t = utime.ticks_us()
                result = f(*args, **kwargs)
                run_times[i] = utime.ticks_diff(utime.ticks_us(), t)
            print('{}() execution times based on {} cycles'.format(func_name, n, (delta2-delta1)/n))
            print('\tbest: %d us'%np.min(run_times))
            print('\tworst: %d us'%np.max(run_times))
            print('\taverage: %d us'%np.mean(run_times))
            print('\tdeviation: +/-%.3f us'%np.std(run_times))            
            return result
        return new_func
    return wrapper

def timeit(f, *args, **kwargs):
    func_name = str(f).split(' ')[1]
    def new_func(*args, **kwargs):
        t = utime.ticks_us()
        result = f(*args, **kwargs)
        print('execution time: ', utime.ticks_diff(utime.ticks_us(), t), ' us')
        return result
    return new_func

__END_OF_DEFS__

# PID module

`ulab` provides a module with a generic and fast implementation of the proportional-integral-derivative ([PID](https://en.wikipedia.org/wiki/Proportional%E2%80%93integral%E2%80%93derivative_controller)) controller. While it is relatively easy to write a PID controller in `python`, the problem with that approach is execution speed: even in thermal applications, where the time scales are not particularly short, running more than one controller simultanously might be difficult, but if the goal is e.g. to control an object mechanically, the task will prove outright impossible. The reason for this slowness of the code execution is the number of floating point operations, where the type-agnostic nature of `python` requires constant type conversion.

`ulab`'s `PID` module implements the controller in C, and for very time-critical applications, it allows the user to take data direclty from the measurement
device (most probably an analogue-to-digital converter), if it supports the buffer protocol. Likewise, data can be output directly to the physical device (in most cases a digital-to-analogue converter), if it can take data from a buffer. This direct access to the device data eliminates the need for data conversion in `python`, and reduces the number of objects that must be passed to the controller loop. In fact, in most cases the controller can be run without moving any data whatsoever, simply by calling a method of the `PID` instance without arguments.

## Background

In general, running a controller loop requires the following steps:

1. Define desired set point
1. Measure the current value of the quantity that is to be controlled
1. Calculate the error, the difference between set point and current value
1. Based on the error's value, calculate the output; this is where the control loop itself is defined
1. Place the calculated output on a physical output device
1. Repeat indefinitely

While the description above seems innocuous, steps 2-5 might require considerable resources, when implemented in an interpreted language.

### Measuring the value of a physical quantity

The quantity that a physical device can measure is not necessarily the quantity that we are interested in, which means that the measured quantity has to undergo some conversion steps before it can be used in the controller loop. 

Take the example of temperature measurements: while there are digital thermometers that will output the temperature directly, in most cases, the temperature is only inferred from a voltage. In the simplest case, a thermistor will form half of a voltage divider connected to a reference voltage $U_{ref}$, and the potential $U_{meas}$ is measured at mid-point. It is obvious that even if the thermistors temperature-resistance relationship $R_{th}(T)$ were to be linear, the measured potential will not be (it does not matter, whether the thermistor is in the upper or lower half of the divider):

\begin{equation}
U_{meas} = U_{ref}\frac{R}{R + R_{th}(T)}
\end{equation}

Most thermistors will have a highly non-linear characteristic curve, thus, converting the potential values to temperature becomes non-trivial, and in the most general case, the conversion must be calculated from the expression above, by inverting $R_{th}(T)$. We can note, however, that any well-behaved function can be expanded in a Taylor series around a point, where the series converges, hence the temperature can be expressed as 

\begin{equation}
T = f(U_{meas}) = 
f(U_0) + 
\left. \frac{df}{dU} \right|_{U_0} (U_{meas} - U_0) + 
\left. \frac{1}{2}\frac{d^2f}{dU^2} \right|_{U_0} (U_{meas} - U_0)^2 + 
\left. \frac{1}{6}\frac{d^3f}{dU^3} \right|_{U_0} (U_{meas} - U_0)^3 + ...
\end{equation}
where the differentials are evaluated at $U_0$.


The coefficients of the expansion are determined by the parameters of the physical system in question, as well as $U_0$. Also note that higher-order expansions are inherently more accurate, but take more time to calculate. Usually, a second-order expansion, or even a linear one is enough, if the expected deviations from $U_0$ are not too large.

### Converting the digital control values to physical control signals

The comments on converting measured values to physically relevant quantities apply, when we want to convert calculated values to physially relevant control signals, i.e., when at the end of the control loop, we want to use the calculated signal to influence the physical drive mechanism: the calculated digital value will be converted to a current, potential etc. by means of a digital-to-analogue converter, and some analogue circuitry that takes an analogue potential as input, and generates a current, or another potential as output, or turns a valve. The relationship between the input potential and the physical control quantity is not necessarily the identity, or even linear, thus, in the general case, we have to express the conversion function as 

\begin{equation}
I_{control} = g(U_{control}) = 
g(U^*) + 
\left. \frac{dg}{dU} \right|_{U^*} (U_{control} - U^*) + 
\left. \frac{1}{2}\frac{d^2g}{dU^2} \right|_{U^*} (U_{control} - U^*)^2 + 
\left. \frac{1}{6}\frac{d^3g}{dU^3} \right|_{U^*} (U_{control} - U^*)^3 + ...
\end{equation}
where the control output is expanded around the potential $U^*$. 

The coefficients of the expansion are determined by the parameters of the physical system in question, as well as $U^*$. Also note that higher-order expansions are inherently more accurate, but take more time to calculate. Usually, a second-order expansion, or even a linear one is enough, if the expected deviations from $U^*$ are not too large.

### Input-output buffers

Conversion between analogue and digital values works not with floating point numbers, which are most probably the results of the controller algorithm, but with integers that are transmitted to/from the converter. A typical transmission sequence will, in general, contain not only the data, but some auxiliary bits, like status, channel number, gain, etc., as shown below. The input/output string $S$ consists of $n + N + m$ bits, where the *D*s are the data bits with the digital values, while the *A*s are the auxiliary bits. The data are embedded in a string, whose auxiliary bits are fixed, and do not change from transmission to transmission.

\begin{equation}
S = \overbrace{A_n A_{n - 1} ... A_1 A_0}^{\mathrm{auxiliary\ bits}}
\overbrace{D_N D_{N - 1} ... D_1 D_0}^{\mathrm{data\ bits}} 
\overbrace{A_m A_{m - 1} ... A_1 A_0}^{\mathrm{auxiliary\ bits}}
\end{equation}

In order to make data exchange between the control algorithm and the peripheral devices transparent and seamless, the `PID` module implements input/output buffers, which not only place the relevant bits at the correct positions in pre-defined strings automatically, but also take care of the conversion of data bits to floating point numbers and vica versa.

Since we are not interested in the sequences $A_n A_{n - 1} ... A_1 A_0$, and $A_m A_{m - 1} ... A_1 A_0$, the data can be recovered by shifting its bits to the right by so many places that $D_0$ reaches a byte boundary, and then masking the result by $2^N - 1$ (this is equivalent to $N$ 1s). In the code, $n$ will be referred to as the `offset`, while $N$ as the `bitdepth`. The exact values should be taken from the data sheets of the ADC or DAC that are used as the interface devices.

### Input-output data converters

We have seen above how the numbers in the control algorithm can be interfaced with the outside world. To facilitate the convension, `ulab`'s `PID` module defines input-output data converters, which hold an input/output buffer as described in the previous section, and the coefficients of the Taylor series expansions, i.e., all information required to turn a string of bits into a floating point number that is the true represenation of the value that we are interested in.

## Methods 

The module implements a class with the following methods

1. [bitdepth](#bitdepth)
1. [offset](#offset)
1. [buffer](#buffer)
1. [evaluate](#evaluate)
1. [parameters](#parameters)
1. [reset](#reset)
1. [series](#series)
1. [step](#step)
1. [float_step](#float_step)
1. [value](#value)

### Instantiating the PID object

The controller can be instantiated by importing the `PID` module, and calling the constructor without arguments. At this point, the object will hold all values required for the operation of the PID loop, and some of these can be displayed by simply printing the controller object: 

In [9]:
%%micropython -unix 1

from ulab import PID

pid = PID.PID()
print(pid)

PID object at 0x7fb58e2053a0
value: 0.000000
last time: 0.000000
steps: 0




The `value` is the current value of the controller's output, `last time` is when the controller's `step` method was called last. Time is measured from the moment of the first step. `steps` is the number of steps the controller has taken since its start.

### bitdepth

This method *gets* or *sets* the bit depth of the input's ADC or the output's DAC. Note that the method does not communicate with the ADC or DAC, it simply changes how the bits in the input and output buffers are interpreted. The method takes 1 or 2 arguments. The first one is the strings `in` or `out`. If this is the only argument supplied (i.e., the method is a *getter*), the method returns the currently active bitdepth setting for that buffer. If the method is called with arguments (i.e., when it is a *setter*), this argument must be an unsigned integer, signifying the number of bits taken by the data in the buffer. 

In [14]:
%%micropython -unix 1

from ulab import PID

pid = PID.PID()
print('initial bit depth: ', pid.bitdepth('in'))
pid.bitdepth('in', 12)
print('new bit depth: ', pid.bitdepth('in'))

initial bit depth:  0
new bit depth:  12




### offset

This method *gets* or *sets* the offset of the input's ADC or the output's DAC stream. Note that the method does not communicate with the ADC or DAC, it simply changes how the bits in the input and output buffers are interpreted. The method takes 1 or 2 arguments. The first one is the strings `in` or `out`. If this is the only argument supplied (i.e., the method is a *getter*), the method returns the currently active offset setting for that buffer. If the method is called with arguments (i.e., when it is a *setter*), this argument must be an unsigned integer, signifying the offset expressed in bits. 

In [7]:
%%micropython -unix 1

from ulab import PID

pid = PID.PID()
print('offset: ', pid.offset('in'))
pid.offset('in', 12)
print('new offset: ', pid.offset('in'))

offset:  176
new offset:  12




### reset

This method takes no arguments, and resets the controller, i.e., zeroes the time, the number of steps, and the integral part.

In [22]:
%%micropython -unix 1

from ulab import PID

pid = PID.PID()
print('initial state of controller')
print(pid)

print()
print('stepping the controller 10 times')
for _ in range(10):
    pid.step()

print()
print('new state of controller:')
print(pid)

print()
print('resetting controller')
pid.reset()

print()
print('new state of controller:')
print(pid)


initial state of controller
PID object at 0x7f0ad62053a0
value: 0.000000
last time: 0.000000
steps: 0

stepping the controller 10 times

new state of controller:
PID object at 0x7f0ad62053a0
value: 0.000000
last time: 0.000000
steps: 10

resetting controller

new state of controller:
PID object at 0x7f0ad62053a0
value: 0.000000
last time: 0.000000
steps: 0




### step

This method takes no arguments, and steps the controller, i.e., takes and converts the data in the inpput buffer, calculates the three terms of the controller's output, and places the data on the output buffer. At the same time, the `value`, `steps`, and `integral` attributes of the controller are updated.  

### float_step

In [50]:
%%micropython -unix 1

from ulab import PID

pid = PID.PID()
pid.parameters('P', 1.0)
pid.parameters('D', 1.0)
print('P: ', pid.parameters('P'))

print(pid.float_step(1.0))
print(pid.float_step(2.0, 0.2))

P:  1.0
2.0
0.75




### parameters



In [42]:
%%micropython -unix 1

from ulab import PID

pid = PID.PID()
print('P: ', pid.parameters('P'))
print('I: ', pid.parameters('I'))
print('D: ', pid.parameters('D'))

print()
pid.parameters('P', 1.0)
pid.parameters('I', 2.0)
pid.parameters('D', 3.0)
print('P: ', pid.parameters('P'))
print('I: ', pid.parameters('I'))
print('D: ', pid.parameters('D'))


P:  0.0
I:  0.0
D:  0.0

P:  1.0
I:  2.0
D:  3.0




## Converting input/output data

### series

The `series` method is again a *getter/setter*, and takes 1 or 3 arguments. The first one is the strings `in` or `out`. If this is the only argument supplied (i.e., the method is a *getter*), the method returns a tuple with the point, around which the function is expanded in a Taylor series, as well as the coefficients of the expansion.

If 3 arguments are supplied, the method becomes a *setter*. The second argument must be a floating point number on which the expansion is centred, while the third argument can be a generic iterable holding the floating point coefficients of the expansion starting with the highest power.

In [37]:
%%micropython -unix 1

from ulab import PID

pid = PID.PID()
# y = x, default
print('Taylor series, input: ', pid.series('in'))

# y = (x - 10) * (x - 10) + 2 * (x - 10) + 3
pid.series('out', 10.0, (1, 2, 3))
print('Taylor series, output: ', pid.series('out'))

Taylor series, input:  (0.0, (1.0, 0.0))
Taylor series, output:  (10.0, (1.0, 2.0, 3.0))




### evaluate

This is a convenience method, which can be used to evaluate the Taylor series at a particular point. Internally, it calls the same function that is called by the controller, whenever it needs to convert its input or output. The method takes two arguments, the designator of the buffer (`in/out`), and the point at which the Taylor expansion is to be evaluated.

In [38]:
%%micropython -unix 1

from ulab import PID

pid = PID.PID()

# y = x, default
print('Taylor series, input: ', pid.series('in'))
print('value at x = 10.0: ', pid.evaluate('in', 10.0))

print()

# y = 2 * x * x + 10.0
pid.series('in', 0.0, (2, 0, 10))
print('Taylor series, input: ', pid.series('in'))

print('value at x = 10.0: ', pid.evaluate('in', 10.0))

Taylor series, input:  (0.0, (1.0, 0.0))
value at x = 10.0:  10.0

Taylor series, input:  (0.0, (2.0, 0.0, 10.0))
value at x = 10.0:  210.0


